<a href="https://colab.research.google.com/github/KBVKarthik/Google-Colaboratory-Notebooks/blob/main/Prediction_Intervals_for_Deep_Learning_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
 

def load_dataset(url):
	dataframe = read_csv(url, header=None)
	values = dataframe.values

	X, y = values[:, :-1], values[:,-1]

	X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, random_state=1)

	scaler = MinMaxScaler()
	scaler.fit(X_train)
	X_train = scaler.transform(X_train)
	X_test = scaler.transform(X_test)
	return X_train, X_test, y_train, y_test
 

def fit_model(X_train, y_train):

	features = X_train.shape[1]
	model = Sequential()
	model.add(Dense(20, kernel_initializer='he_normal', activation='relu', input_dim=features))
	model.add(Dense(5, kernel_initializer='he_normal', activation='relu'))
	model.add(Dense(1))

	opt = Adam(learning_rate=0.01, beta_1=0.85, beta_2=0.999)
	model.compile(optimizer=opt, loss='mse')

	model.fit(X_train, y_train, verbose=0, epochs=300, batch_size=16)
	return model
 

def fit_ensemble(n_members, X_train, X_test, y_train, y_test):
	ensemble = list()
	for i in range(n_members):

		model = fit_model(X_train, y_train)

		yhat = model.predict(X_test, verbose=0)
		mae = mean_absolute_error(y_test, yhat)
		print('>%d, MAE: %.3f' % (i+1, mae))

		ensemble.append(model)
	return ensemble
 

def predict_with_pi(ensemble, X):

	yhat = [model.predict(X, verbose=0) for model in ensemble]
	yhat = asarray(yhat)

	interval = 1.96 * yhat.std()
	lower, upper = yhat.mean() - interval, yhat.mean() + interval
	return lower, yhat.mean(), upper
 

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
X_train, X_test, y_train, y_test = load_dataset(url)

n_members = 30
ensemble = fit_ensemble(n_members, X_train, X_test, y_train, y_test)

newX = asarray([X_test[0, :]])
lower, mean, upper = predict_with_pi(ensemble, newX)
print('Point prediction: %.3f' % mean)
print('95%% prediction interval: [%.3f, %.3f]' % (lower, upper))
print('True value: %.3f' % y_test[0])

>1, MAE: 2.896
>2, MAE: 2.302
>3, MAE: 2.374
>4, MAE: 6.622
>5, MAE: 2.486
>6, MAE: 2.322
>7, MAE: 2.365
>8, MAE: 2.517
>9, MAE: 2.855
>10, MAE: 2.365
>11, MAE: 2.303
>12, MAE: 2.631
>13, MAE: 2.310
>14, MAE: 2.704
>15, MAE: 2.527
>16, MAE: 2.768
>17, MAE: 2.478
>18, MAE: 2.309
>19, MAE: 2.470
>20, MAE: 6.621
>21, MAE: 2.954
>22, MAE: 2.261
>23, MAE: 2.528
>24, MAE: 2.406
>25, MAE: 2.498
>26, MAE: 2.456
>27, MAE: 2.554
>28, MAE: 2.485
>29, MAE: 2.307
>30, MAE: 2.664
Point prediction: 30.377
95% prediction interval: [24.701, 36.052]
True value: 28.200
